## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 4.5 MB/s 
     |████████████████████████████████| 10.3 MB 90.8 MB/s 
     |████████████████████████████████| 7.6 MB 76.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.3 MB 73.3 MB/s 
     |████████████████████████████████| 5.5 MB 76.1 MB/s 
     |████████████████████████████████| 1.9 MB 74.3 MB/s 
     |████████████████████████████████| 451 kB 87.7 MB/s 
     |████████████████████████████████| 182 kB 54.0 MB/s 
     |████████████████████████████████| 168 kB 80.7 MB/s 
     |████████████████████████████████| 182 kB 82.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 168 kB 86.3 MB/s 
     |████████████████████████████████| 166 kB 91.6 MB/s 
     |████████████████████████████████| 166 kB 92.1 MB/s 
     |█████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test1.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

1    648
3    648
2    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

{'mcc': 0.9948462353926073, 'f1': array([0.99186992, 1.        , 1.        , 0.99354839]), 'acc': 0.9961389961389961, 'recall': array([1.        , 1.        , 1.        , 0.98717949]), 'precision': array([0.98387097, 1.        , 1.        , 1.        ]), 'eval_loss': 0.013934168249375723}
results accuracy [0.7576923076923077, 0.8192307692307692, 0.8648648648648649, 0.9111969111969112, 0.9613899613899614, 0.972972972972973, 0.9922779922779923, 0.9884169884169884, 0.9961389961389961]
Mean-Precision: 0.918242418242418
results f1:  [array([0.84671533, 0.71755725, 0.8503937 , 0.608     ]), array([0.87394958, 0.77966102, 0.88888889, 0.73381295]), array([0.94736842, 0.86516854, 0.89285714, 0.78195489]), array([0.98591549, 0.85714286, 0.95714286, 0.81818182]), array([0.97931034, 0.94308943, 0.97709924, 0.94117647]), array([0.99236641, 0.95934959, 0.98461538, 0.95522388]), array([0.99159664, 0.99224806, 1.        , 0.98333333]), array([0.99270073, 0.97777778, 1.        , 0.98484848]), array([0.

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 1.0, 'f1': array([1., 1., 1., 1.]), 'acc': 1.0, 'recall': array([1., 1., 1., 1.]), 'precision': array([1., 1., 1., 1.]), 'eval_loss': 0.0025391450734318687}
results accuracy [0.7576923076923077, 0.8192307692307692, 0.8648648648648649, 0.9111969111969112, 0.9613899613899614, 0.972972972972973, 0.9922779922779923, 0.9884169884169884, 0.9961389961389961, 1.0]
Mean-Precision: 0.9264181764181763
results f1:  [array([0.84671533, 0.71755725, 0.8503937 , 0.608     ]), array([0.87394958, 0.77966102, 0.88888889, 0.73381295]), array([0.94736842, 0.86516854, 0.89285714, 0.78195489]), array([0.98591549, 0.85714286, 0.95714286, 0.81818182]), array([0.97931034, 0.94308943, 0.97709924, 0.94117647]), array([0.99236641, 0.95934959, 0.98461538, 0.95522388]), array([0.99159664, 0.99224806, 1.        , 0.98333333]), array([0.99270073, 0.97777778, 1.        , 0.98484848]), array([0.99186992, 1.        , 1.        , 0.99354839]), array([1., 1., 1., 1.])]
results precision:  [array([0.81690141, 0.6351

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7576923076923077, 0.8192307692307692, 0.8648648648648649, 0.9111969111969112, 0.9613899613899614, 0.972972972972973, 0.9922779922779923, 0.9884169884169884, 0.9961389961389961, 1.0]
Mean-accuracy: 0.9264181764181763
results f1:  [array([0.84671533, 0.71755725, 0.8503937 , 0.608     ]), array([0.87394958, 0.77966102, 0.88888889, 0.73381295]), array([0.94736842, 0.86516854, 0.89285714, 0.78195489]), array([0.98591549, 0.85714286, 0.95714286, 0.81818182]), array([0.97931034, 0.94308943, 0.97709924, 0.94117647]), array([0.99236641, 0.95934959, 0.98461538, 0.95522388]), array([0.99159664, 0.99224806, 1.        , 0.98333333]), array([0.99270073, 0.97777778, 1.        , 0.98484848]), array([0.99186992, 1.        , 1.        , 0.99354839]), array([1., 1., 1., 1.])]
results precision:  [array([0.81690141, 0.63513514, 0.93103448, 0.66666667]), array([0.89655172, 0.86792453, 0.88888889, 0.66233766]), array([0.91836735, 0.83695652, 0.90909091, 0.82539683]), array([1.        , 0

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.957314,0.898054,0.966862,0.890562
Average recall,0.963668,0.926348,0.944848,0.873302
Average F1-Score,0.960179,0.909199,0.955100,0.880008


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test1_bert.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   69.79 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 74.26 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   43.93 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.8874734607218684
Precision D:  0.3378995433789954
Precision I:  0.3333333333333333
Precision P:  0.48253275109170307
Recall B:  0.7586206896551724
Recall D:  0.7789473684210526
Recall I:  0.03333333333333333
Recall P:  0.5594936708860759
F1-score B:  0.8180039138943248
F1-score D:  0.4713375796178343
F1-score I:  0.0606060606060606
F1-score P:  0.5181711606096131


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.2340998302797192


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test1.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

2    743
0    743
1    743
3    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.7630452301326547, 'f1': array([0.90163934, 0.80446927, 0.89361702, 0.71428571]), 'acc': 0.8221476510067114, 'recall': array([0.91666667, 0.84705882, 0.81818182, 0.72368421]), 'precision': array([0.88709677, 0.76595745, 0.984375  , 0.70512821]), 'eval_loss': 0.507691784908897}
results accuracy [0.8221476510067114]
Mean-accuracy: 0.8221476510067114
results f1:  [array([0.90163934, 0.80446927, 0.89361702, 0.71428571])]
results precision:  [array([0.88709677, 0.76595745, 0.984375  , 0.70512821])]
results recall:  [array([0.91666667, 0.84705882, 0.81818182, 0.72368421])]


...

{'mcc': 0.9864809921954237, 'f1': array([0.98360656, 0.98901099, 0.9929078 , 0.99328859]), 'acc': 0.98989898989899, 'recall': array([0.96774194, 0.98901099, 1.        , 1.        ]), 'precision': array([1.        , 0.98901099, 0.98591549, 0.98666667]), 'eval_loss': 0.0450251388706659}
results accuracy [0.8221476510067114, 0.9295302013422819, 0.9528619528619529, 0.9764309764309764, 0.9831649831649831, 0.9966329966329966, 0.9966329966329966, 0.9966329966329966, 0.98989898989899]
Mean-accuracy: 0.9604370827338761
results f1:  [array([0.90163934, 0.80446927, 0.89361702, 0.71428571]), array([0.95808383, 0.93333333, 0.953125  , 0.87417219]), array([0.97005988, 0.92857143, 0.99371069, 0.91025641]), array([0.98550725, 0.97109827, 1.        , 0.94736842]), array([0.97478992, 0.98529412, 1.        , 0.96969697]), array([1.        , 1.        , 0.9929078 , 0.99319728]), array([1.        , 0.99212598, 1.        , 0.99363057]), array([1.        , 0.99428571, 1.        , 0.99065421]), array([0.98360

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.9910564172794819, 'f1': array([1.        , 0.98461538, 1.        , 0.98795181]), 'acc': 0.9932659932659933, 'recall': array([1.        , 0.96969697, 1.        , 1.        ]), 'precision': array([1.        , 1.        , 1.        , 0.97619048]), 'eval_loss': 0.010101981853183947}
results accuracy [0.8221476510067114, 0.9295302013422819, 0.9528619528619529, 0.9764309764309764, 0.9831649831649831, 0.9966329966329966, 0.9966329966329966, 0.9966329966329966, 0.98989898989899, 0.9932659932659933]
Mean-accuracy: 0.9637199737870878
results f1:  [array([0.90163934, 0.80446927, 0.89361702, 0.71428571]), array([0.95808383, 0.93333333, 0.953125  , 0.87417219]), array([0.97005988, 0.92857143, 0.99371069, 0.91025641]), array([0.98550725, 0.97109827, 1.        , 0.94736842]), array([0.97478992, 0.98529412, 1.        , 0.96969697]), array([1.        , 1.        , 0.9929078 , 0.99319728]), array([1.        , 0.99212598, 1.        , 0.99363057]), array([1.        , 0.99428571, 1.        , 0.99

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.8221476510067114, 0.9295302013422819, 0.9528619528619529, 0.9764309764309764, 0.9831649831649831, 0.9966329966329966, 0.9966329966329966, 0.9966329966329966, 0.98989898989899, 0.9932659932659933]
Mean-accuracy: 0.9637199737870878
results f1:  [array([0.90163934, 0.80446927, 0.89361702, 0.71428571]), array([0.95808383, 0.93333333, 0.953125  , 0.87417219]), array([0.97005988, 0.92857143, 0.99371069, 0.91025641]), array([0.98550725, 0.97109827, 1.        , 0.94736842]), array([0.97478992, 0.98529412, 1.        , 0.96969697]), array([1.        , 1.        , 0.9929078 , 0.99319728]), array([1.        , 0.99212598, 1.        , 0.99363057]), array([1.        , 0.99428571, 1.        , 0.99065421]), array([0.98360656, 0.98901099, 0.9929078 , 0.99328859]), array([1.        , 0.98461538, 1.        , 0.98795181])]
results precision:  [array([0.88709677, 0.76595745, 0.984375  , 0.70512821]), array([0.94117647, 0.93333333, 0.953125  , 0.89189189]), array([0.95294118, 0.92857143, 

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.974977,0.955327,0.991092,0.938382
Average recall,0.980000,0.961756,0.975722,0.936821
Average F1-Score,0.977369,0.958280,0.982627,0.937450
